## Explore dataset

In [6]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter

In [ ]:
# terminal stuff
# !uv sync
# !source .venv/bin/activate
# !uv run -- spacy download es_core_news_md

/Users/adb/stuff/gitclones/ConvoKit_Disputes/.venv/bin/python3: No module named pip


In [8]:
# data/alldyads.csv
# data/headers.csv
# Replace 'path/to/your_data.csv' with the actual path to your CSV file
df = pd.read_csv('../data/alldyads.csv', header=0)

# Quick preview of the DataFrame
print(df.head())

# dyads_df = pd.read_csv('../data/alldyads.csv')
headers_df = pd.read_csv('../data/headers.csv')

   b_country  b_RankBuy_1  b_RankBuy_2  b_RankBuy_3  b_RankBuy_4  b_Tact_1  \
0      187.0         80.0         12.0          5.0          3.0       1.0   
1      187.0         40.0         10.0         31.0         19.0       3.0   
2      187.0         40.0         10.0         30.0         20.0       2.0   
3        NaN          NaN          NaN          NaN          NaN       NaN   
4        NaN          NaN          NaN          NaN          NaN       NaN   

   b_Tact_2  b_Tact_3  b_Tact_4  b_Tact_5  ...  s_AI4me  s_Ai4me-why  \
0       4.0       5.0       NaN       1.0  ...      NaN          NaN   
1       1.0       2.0       NaN       1.0  ...      NaN          NaN   
2       4.0       4.0       NaN       3.0  ...      NaN          NaN   
3       NaN       NaN       NaN       NaN  ...      NaN          NaN   
4       NaN       NaN       NaN       NaN  ...      NaN          NaN   

   s_Use emotions  s_AI4u  s_PlayAI.1  s_AI-cues.1  is_AI  \
0             NaN     NaN         NaN

In [9]:
headers_df = headers_df.T #transpose
# Replace any empty strings ("") with NaN across all columns
df = df.replace('', np.nan)


In [10]:
numeric_cols = [
    'b_country',
    'b_RankBuy_1','b_RankBuy_2','b_RankBuy_3','b_RankBuy_4',
    's_country',
    's_RankSell_1','s_RankSell_2','s_RankSell_3','s_RankSell_4'
    # ... Add more columns you know should be numeric
]

for col in numeric_cols:
    # coerce errors turns invalid parsing into NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [11]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 50 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   b_country       2510 non-null   float64
 1   b_RankBuy_1     2543 non-null   float64
 2   b_RankBuy_2     2543 non-null   float64
 3   b_RankBuy_3     2543 non-null   float64
 4   b_RankBuy_4     2543 non-null   float64
 5   b_Tact_1        2160 non-null   float64
 6   b_Tact_2        2160 non-null   float64
 7   b_Tact_3        2160 non-null   float64
 8   b_Tact_4        2143 non-null   float64
 9   b_Tact_5        2160 non-null   float64
 10  b_Tact_6        2160 non-null   float64
 11  b_Tact_7        2160 non-null   float64
 12  b_Tact_8        2160 non-null   float64
 13  b_Tact_9        2143 non-null   float64
 14  b_Tact_10       2160 non-null   float64
 15  b_PlayAI        405 non-null    float64
 16  b_AI-cues       400 non-null    object 
 17  b_AI4me         404 non-null    f

In [12]:
print(df['is_AI'].value_counts(dropna=False))


is_AI
False    2116
True      775
Name: count, dtype: int64


In [13]:
# Example: Mean of the rank columns
rank_means = df[[
    'b_RankBuy_1','b_RankBuy_2','b_RankBuy_3','b_RankBuy_4',
    's_RankSell_1','s_RankSell_2','s_RankSell_3','s_RankSell_4'
]].mean()
print(rank_means)


b_RankBuy_1     57.086905
b_RankBuy_2     13.838380
b_RankBuy_3     16.721589
b_RankBuy_4     12.353126
s_RankSell_1    31.568228
s_RankSell_2    15.039308
s_RankSell_3    34.503055
s_RankSell_4    18.889409
dtype: float64


In [14]:
nlp = spacy.load("en_core_web_sm")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [20]:
!uv pip install pip
!uv run -- spacy download en_core_web_sm

Using Python 3.11.11 environment at: /Users/adb/stuff/gitclones/ConvoKit_Disputes/.venv
Audited 1 package in 9ms
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [21]:
spacy.load('en_core_web_sm')

In [22]:
def lemmatize_text(text):
    # If the text is NaN, return as-is
    if pd.isna(text):
        return text
    
    doc = nlp(text)
    # Filter out stopwords, punctuation, and spaces, then lemmatize
    tokens = [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and not token.is_punct and not token.is_space
    ]
    return " ".join(tokens)


In [23]:
df['lemmatized_chat'] = df['formattedChat'].apply(lemmatize_text)

# Quick look at the new column
print(df[['formattedChat', 'lemmatized_chat']].head())


NameError: name 'nlp' is not defined